In [1]:
from sklearn.datasets import fetch_mldata
from sklearn import tree
import random as rand
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import BaggingRegressor
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import sys
import pickle

## 1.- Get the MNIST dataset, select one kind of model and train multiple instances of the same model in this dataset (say, from 1 to 100).

In [2]:
mnist = fetch_mldata('MNIST original')
x_train, x_test, y_train, y_test = train_test_split(
    mnist.data, mnist.target, test_size=0.4, random_state=0)


In [3]:
#Considering decision tree to be the base algorithm 
class ensemble():
    def __init__(self):
        pass

#     def train_data_LR(self,train, target):
#         n_samples = len(train.data)
#         data = train.reshape((n_samples, -1))
#         clf = LinearRegression()
#         return clf.fit(data, target)
    
#     def svm(self,train, target):
#         n_samples = len(train.data)
#         data = train.reshape((n_samples, -1))
#         clf = SVC(kernel='rbf', probability=True)
#         return clf.fit(data, target)
    
    def train_data_DT(self, train, target):
        n_samples = len(train.data)
        data = train.reshape((n_samples, -1))
        clf = tree.DecisionTreeClassifier()
        return clf.fit(data, target)
    
    def bagging(self, clf, train, target):
        accuracy = []
        for i in range(2,4):
            print("Number of members = ",i)
            baglfy=BaggingClassifier(base_estimator=clf,n_estimators=i,max_samples=len(train))
            baglfy=baglfy.fit(x_train,y_train)
            bag_pred=baglfy.predict(x_test)
            print("Accuracy = ",accuracy_score(y_test, bag_pred))
            accuracy.append(accuracy_score(y_test, bag_pred))
        return baglfy


In [8]:
obj2 = ensemble()
clf2 = obj2.train_data_DT(x_train,y_train)
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(clf2, open(filename, 'wb'))

In [9]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

## 2.- With the trained models, make ensembles of [2, 3, ..., 100] members, evaluate the testing accuracy of each ensemble, and estimate the uncertainty of their outputs using a histogram or average of softmax outputs (depending on the model).

In [5]:
n_runs = 4
baggingClfs = []
clf = tree.DecisionTreeClassifier()
for i in range(2,n_runs):
    print("Number of members = ",i)
    baglfy=BaggingClassifier(base_estimator=clf,n_estimators=i,max_samples=len(x_train),bootstrap=True, n_jobs=-1)
    baglfy=baglfy.fit(x_train,y_train)
    baggingClfs.append(baglfy)
    
filename = 'finalized_model1.sav'
pickle.dump(baggingClfs, open(filename, 'wb'))

Number of members =  2
Number of members =  3


In [6]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
print(loaded_model)

[BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=42000, n_estimators=2, n_jobs=-1, oob_score=False,
         random_state=None, verbose=0, warm_start=False), BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstr

In [21]:
accuracy = []
for i in range(2,n_runs):
    uncer = loaded_model.predict_proba(x_train)
    print("Uncertainity = ", uncer)
    print("Accuracy = ",accuracy_score(y_test, bag_pred))
    accuracy.append(accuracy_score(y_test, bag_pred))

Uncertainity =  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Uncertainity =  

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
# a = clf.predict_proba(x_train)


## 3.- Does accuracy and uncertainty improve by having more members in each ensemble?

## 4.- Using a single ensemble of your choosing (you define the number of members), find the misclassified examples in the test set and analyze the uncertainty of those examples. Can the uncertainty explain why those examples are misclassified? Give examples and a complete analysis. 

Intuitively, this can be seen as a simple question. If we have
several models trained on the same data, the model's
parameters won't be the same. Some parameters might be
very close, some will be equal, and some will be radically
different.
These variations on model parameters can be encoded as a
probability distribution. Bayesian statistics can be used to
estimate these distributions P(wjD)

## 5.- Reproduce Figure one from the "Simple and Scalable Predictive UncertaintyEstimation using Deep Ensembles" paper (attached) using a random forest, computing regression uncertainty as the variance of the estimates across trees. The authors mention their metodology in Section 3.2, and one can easily reproduce the training set as its a sample of y = x^3 + noise, where noise is Gaussian with zero mean and 3^2 variance, and x is in the range [-4, 4]. The key idea of those figures is to show that uncertainty grows for values outside of the range [-4, 4], more specifically, in the range [-6, -4] and [4, 6]. To produce the figure, you can plot the mean and make confidence bands with size of square root of variance.Remember that I expect a good report of results, not just concentrating on the code, but also on scientific analysis of your results and explanations.